In [21]:
import torch
import torch.nn as nn

from Layers import GPTBlock
import tiktoken

In [22]:

def generate_text_simple(model, idx, max_new_tokens, context_size):

    '''
    model = the GPT model of the system
    idx = the output of the GPT model (batch_size, num_tokens)
    max_new_tokens = the user defined parameter, how many new tokens should be generated
    context_size = context size as per model config (e.g 1024 in case of GPT)
    '''

    for _ in range(max_new_tokens):


        idx_cond = idx[:,-context_size:] # if num_tokens > context_size :, take only inputs corresponding to context_size from last 

        # Get the predictions:
        with torch.no_grad():
            logits = model(idx_cond)

        # Extract the last row
        logits  = logits[:,-1,:] # (batch, n_tokens, vocab_size) --> (batch, vocab_size)

        # Apply softmax to get probabilities
        probs = torch.softmax(logits, dim=-1) #(batch, vocab_size)


        # Get the ID of vocab entry with highest prob. value
        idx_next = torch.argmax(probs, dim=-1, keepdim=True) #(batch,1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1) #(batch, num_tokens+1)

    return idx

_____________________

In [23]:
tokenizer = tiktoken.get_encoding("gpt2")
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0) #A
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [24]:
cs = 5
idx_cond = encoded_tensor[:,-cs:] # if num_tokens > context_size :, take only inputs corresponding to context_size from last 
idx_cond


tensor([[15496,    11,   314,   716]])

In [25]:
GPT_CONFIG_124M = {
    "vocab_size" : 50527,  # total no of unique tokens
    "context_length" : 1024,  # seq length or context length
    "emb_dim" : 768, # embedding dim, hidden dim, d_model
    "n_heads" : 12, # number of attention heads in MHA
    "n_layers" : 12, # number of transformer layers
    "drop_rate" : 0.1, # dropout rate
    "qkv_bias" : False # Query Key Value 
}

In [26]:
model = GPTBlock(GPT_CONFIG_124M)

In [27]:
model.eval() #A
out = generate_text_simple(
model=model,
idx=encoded_tensor,
max_new_tokens=6,
context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,    11,   314,   716, 47124,  9319,  6360,  5580, 42286, 38902]])
Output length: 10


In [28]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am thirstylowcludingikiideshow 289


Output is gibberish as model is not trained properly.

________________________________________________

In [29]:
tokenizer = tiktoken.get_encoding("gpt2")
start_context = "Harry potter is an amazing novel series"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0) #A
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [18308, 1787, 353, 318, 281, 4998, 5337, 2168]
encoded_tensor.shape: torch.Size([1, 8])


In [30]:
context_size = 5
out_2 = encoded_tensor[:,-context_size:] # if num_tokens > context_size :, take only inputs corresponding to context_size from last 
out_2

tensor([[ 318,  281, 4998, 5337, 2168]])